In [ ]:
import os
import json
import pandas as pd
from google.colab import drive
from matplotlib import pyplot as plt

# Change working directory to Google Drive folder containg final project files
wd_path = '/content/drive/MyDrive/2024 FALL/INFO 628 - Data Librarianship/final_project'
os.chdir(wd_path)

# Function to pull the earliest date from an array of concert information
def firstConcertDate(concert_array):
    concert_dates = []

    for concert in concert_array:
        concert_dates.append(concert['Date'])

    concert_dates.sort()
    earliest_date = concert_dates[0]

    return earliest_date

In [ ]:
# Import New York Phil Performance History from JSON file to a dictionary
perf_history = {}

with open('ny_phil_performance_history_complete.json', 'r') as json_file:
    perf_history = json.load(json_file)

perf_history_records = perf_history['programs']

In [ ]:
# Convert dictionary from JSON to a DataFrame
perf_df = pd.DataFrame(perf_history_records)

# Create new DataFrame by exploding works column, creating a new row for each
# work on a program
perf_df_works_exploded = (perf_df
                          .explode('works')
                          .reset_index()
                          )

# Split array of 'works' info into multiple columns and drop redundant fields
works_and_programs = perf_df_works_exploded[['programID', 'works']]
works_dict = works_and_programs.to_dict(orient='records')
works_df = (pd.json_normalize(works_dict, sep='_')
            .drop(['programID', 'works','works_soloists', 'works_conductorName'], axis=1)
            .fillna('')
            )

# Aggregate all 'movement' information into 'works_movement' column
works_df['works_movement__'] = works_df[['works_movement__', 'works_movement_em']].agg(' '.join, axis=1).str.strip()
works_df['works_movement'] = works_df[['works_movement', 'works_movement__']].agg(' '.join, axis=1).str.strip()

# Aggregate all 'workTitle' information into 'works_workTitle' column
works_df['works_workTitle__'] = works_df[['works_workTitle__', 'works_workTitle_em']].agg(' '.join, axis=1).str.strip()
works_df['works_workTitle'] = works_df[['works_workTitle', 'works_workTitle__']].agg(' '.join, axis=1).str.strip()

# Change column names to singular/lowercase
works_df_cleaned = (works_df
                    .rename(columns={
                        'works_ID' : "work_id",
                        "works_composerName" : "work_composer_name",
                        "works_workTitle" : "work_title",
                        "works_movement" : "work_movement"})
                    )

# Concatonate the exploded/cleaned 'performance' and 'works' DataFrames,
# renaming 'programID' to match the new naming convention
work_history_df = (pd
                   .concat([perf_df_works_exploded, works_df_cleaned], axis=1)
                   .rename(columns={'programID' : 'program_id'})
                   )

# Create a column ('date') that specifies the first date the work was
# performed on a concert for the associated program
work_history_df['date'] = work_history_df['concerts'].apply(firstConcertDate)

# Remove movement information from work_id
work_history_df['work_id'] = work_history_df['work_id'].apply(lambda x: x.split('*')[0])

# Remove rows with no 'work_id'; remove intermission rows ('work_id' = '0*');
# set index to be a combination of 'work_id' and 'program_id'; reorder and
# select relevant columns.
work_history_df_cleaned = (
    work_history_df[(work_history_df['work_id'] != '') & (work_history_df['work_id'] != '0') & (work_history_df['work_id'] != '')]
    .set_index(['work_id', 'program_id'])
    .sort_values(['work_id', 'date'], ascending=[True, True])
    .loc[:,['date', 'season', 'work_title','work_movement','work_composer_name']]
    )

work_history_df_cleaned.tail(40)

date   season  \
work_id program_id                                  
996     548         1929-02-16T05:00:00Z  1928-29   
        5357        1955-03-24T05:00:00Z  1954-55   
        3172        1964-02-06T05:00:00Z  1963-64   
        6701        1979-05-17T04:00:00Z  1978-79   
        1520        1982-04-29T04:00:00Z  1981-82   
        5891        1982-05-04T04:00:00Z  1981-82   
        9629        2007-01-14T05:00:00Z  2006-07   
        9760        2007-02-18T05:00:00Z  2006-07   
        10091       2007-04-15T04:00:00Z  2006-07   
        10374       2010-02-28T05:00:00Z  2009-10   
        10374       2010-02-28T05:00:00Z  2009-10   
        10375       2010-04-11T04:00:00Z  2009-10   
        10376       2010-05-09T04:00:00Z  2009-10   
        13646       2016-06-02T04:00:00Z  2015-16   
        13895       2016-06-22T04:00:00Z  2015-16   
        14759       2017-06-28T04:00:00Z  2016-17   
9965    10003       2009-05-30T04:00:00Z  2008-09   
9975    10092       1993-10-14T04:00:00Z  1993-94   
9976    5710        1977-05-20T04:00:00Z  1976-77   
        10141       1977-05-24T04:00:00Z  1976-77   
        2689        1977-06-10T04:00:00Z  1976-77   
9977    9234        2001-10-12T04:00:00Z  2001-02   
9978    7213        1916-02-26T05:00:00Z  1915-16   
9979    3029        1916-11-08T05:00:00Z  1916-17   
998     3719        1917-01-09T05:00:00Z  1916-17   
9983    5655        1920-02-06T05:00:00Z  1919-20   
        8774        1926-01-14T05:00:00Z  1925-26   
        5642        1941-01-22T05:00:00Z  1940-41   
        6699        1970-06-05T04:00:00Z  1969-70   
9984    9756        2006-01-22T05:00:00Z  2005-06   
9985    9787        1946-05-10T04:00:00Z  1945-46   
9986    4057        1950-12-16T05:00:00Z  1950-51   
        2133        1956-12-22T05:00:00Z  1956-57   
        53          1957-12-21T05:00:00Z  1957-58   
        922         1958-12-21T05:00:00Z  1958-59   
        1813        1959-12-20T05:00:00Z  1959-60   
9987    474         1968-06-19T04:00:00Z  1967-68   
9988    4198        1978-12-23T05:00:00Z  1978-79   
999     292         1954-12-23T05:00:00Z  1954-55   
9990    10003       2009-05-30T04:00:00Z  2008-09   

                                                           work_title  \
work_id program_id                                                      
996     548         CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        5357        CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        3172        CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        6701        CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        1520        CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        5891        CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        9629        CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        9760        CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        10091       CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        10374       CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        10374       CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        10375       CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        10376       CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        13646       CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        13895       CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
        14759       CONCERTO, VIOLIN, F MAJOR, RV 293 (AUTUMN FROM...   
9965    10003                             GOODBYE JOHN (ARR. Musiker)   
9975    10092       ORFEO ED EURIDICE (ARR. FLUTE AND PIANO) (ARR....   
9976    5710                              WALTZ (UNSPECIFIED) (ORCH.)   
        10141                             WALTZ (UNSPECIFIED) (ORCH.)   
        2689                              WALTZ (UNSPECIFIED) (ORCH.)   
9977    9234        FUNERAL MARCH OF A MARIONETTE (BRASS) (ARR. Un...   
9978    7213                              SHEPHERD'S HEY 

In [ ]:
first_performance_df = (work_history_df_cleaned
                        .groupby('work_id')
                        .first()
                        .sort_values(by='date'))

first_per_season = first_performance_df.season.value_counts()

first_per_season


,count
season,
1945-46,323
2015-16,320
2013-14,263
1900-01,197
2022-23,190
...,...
1869-70,10
1864-65,9
1849-50,7
